# Archivo a archivo
Este apartado tiene como propósito presentar el estudio detallado de cada archivo existente dentro del código Rust del paquete go3, con el objetivo de explicar su funcionamiento clave.

## Código 1 - libs.rs
Este archivo actúa como una capa intermedia que permite la comunicación entre Python y Rust. Por lo tanto, se deduce que la implementación de las funcionalidades principales del paquete está desarrollada completamente en Rust, y luego es expuesta a Python mediante _PyO3_, el cual permite invocarlas utilizando la sintaxis del lenguaje de destino.

In [ ]:
// En esta sección se realiza la envoltura de los archivos Rust con pyo3 estableciendolo en
// 3 pasos:
use pyo3::prelude::*;           //Rust comprende Python (viceversa).
use pyo3::types::PyModule;      //Rust entiende el concepto de módulo de Python.
use pyo3::wrap_pyfunction;      //Python invoca código de Rust.

// Se solicita la carga de tres archivo existentes en el mismo directorio en el que se
// encuentra el archivo actual.
pub mod go_loader;              // Lectura de archivos de entrada, relaciones e IC.
pub mod go_ontology;            // --
pub mod go_semantic;            // --

// Dentro de cada uno de estos archivos mencionados previamente, se solicita la disponibilidad
// de las funciones que se mencionan a continuación.
use go_loader::{load_go_terms, load_gaf, build_term_counter};
use go_ontology::{get_term_by_id, ancestors, common_ancestor, deepest_common_ancestor};
use go_semantic::{
    term_ic,
    semantic_similarity,
    termset_similarity,
    batch_similarity,
    compare_genes,
    compare_gene_pairs_batch,
    gene_distance_matrix,
    tsne_genes,
    umap_genes,
    plot_embedding,
    plot_tsne_genes,
    plot_umap_genes,
    set_num_threads,
};

// En esta sección se realiza la envoltura de todas las componentes invocadas anteriormente
// de forma en que estas sean reconocidad como modulo de Python.
#[pymodule]
fn go3(_py: Python, m: &Bound<'_, PyModule>) -> PyResult<()> {
    m.add_function(wrap_pyfunction!(load_go_terms, m)?)?;
    m.add_function(wrap_pyfunction!(load_gaf, m)?)?;
    m.add_function(wrap_pyfunction!(build_term_counter, m)?)?;

    m.add_function(wrap_pyfunction!(get_term_by_id, m)?)?;
    m.add_function(wrap_pyfunction!(ancestors, m)?)?;
    m.add_function(wrap_pyfunction!(common_ancestor, m)?)?;
    m.add_function(wrap_pyfunction!(deepest_common_ancestor, m)?)?;

    m.add_function(wrap_pyfunction!(set_num_threads, m)?)?;
    m.add_function(wrap_pyfunction!(term_ic, m)?)?;
    m.add_function(wrap_pyfunction!(semantic_similarity, m)?)?;
    m.add_function(wrap_pyfunction!(termset_similarity, m)?)?;
    m.add_function(wrap_pyfunction!(batch_similarity, m)?)?;
    m.add_function(wrap_pyfunction!(compare_genes, m)?)?;
    m.add_function(wrap_pyfunction!(compare_gene_pairs_batch, m)?)?;
    m.add_function(wrap_pyfunction!(gene_distance_matrix, m)?)?;
    m.add_function(wrap_pyfunction!(tsne_genes, m)?)?;
    m.add_function(wrap_pyfunction!(umap_genes, m)?)?;
    m.add_function(wrap_pyfunction!(plot_embedding, m)?)?;
    m.add_function(wrap_pyfunction!(plot_tsne_genes, m)?)?;
    m.add_function(wrap_pyfunction!(plot_umap_genes, m)?)?;

    m.add_class::<go_ontology::PyGOTerm>()?;
    m.add_class::<go_loader::GAFAnnotation>()?;
    m.add_class::<go_loader::TermCounter>()?;

    Ok(())                                                          // Indicador de exito en la exportación de funciones Rust a Python.
}

## Código 2 - go_loader.rs
Archivo encargado del procesamiento de los archivos .obo y gaf para la construcción de las relaciones jerarquicas entre terminos, relaciones gen a termino y el cálculo de IC de cada término existente.

In [ ]:
// ---------------- Librerias.
use pyo3::prelude::*;                                               // Rust comprende Python (viceversa).
use once_cell::sync::OnceCell;                                      // Implementación de variables globales de una sola carga.
use rustc_hash::{FxHashMap as HashMap, FxHashSet as HashSet};       // Invocación de tablas hash.
use std::io::{BufReader, BufRead};                                  // Lectura eficiente de archivos.
use std::fs::File;                                                  // Apertura y manejo de archivos.
use parking_lot::RwLock;                                            // Administración de hilos por semaforo.
use std::path::Path;                                                // Administración de rutas.
use std::fs;                                                        // Administración de entradas y salidas.
use reqwest::blocking::get;                                         // Peticiones para obtención de recursos en internet.
use rayon::prelude::*;                                              // Automatización del uso de todos los nucleos disponibles de un dispositivo.


// ---------------- Componentes globales.
// Declaración de entidades estáticas en memoria y global en toda la ejecución del código;
// cabe señalar que estos son persistentes mientras el código se ejecuta y estos poseen la
// capacidad de ser leidos por varios threads con bloqueo si se requiere escribir.
use crate::go_ontology::{GOTerm, PyGOTerm, collect_ancestors, get_terms_or_error};
pub static GO_TERMS_CACHE: OnceCell<RwLock<HashMap<String, GOTerm>>> = OnceCell::new();                 // Hash entre el GO_ID y su información.
pub static GENE2GO_CACHE: OnceCell<RwLock<HashMap<String, Vec<String>>>> = OnceCell::new();             // Hash entre un gen y sus términos GO relacionados.
pub static ANCESTORS_CACHE: OnceCell<RwLock<HashMap<String, HashSet<String>>>> = OnceCell::new();       // Hash entre un GO_ID y sus respectivos ancestros.
pub static DCA_CACHE: OnceCell<RwLock<HashMap<(String, String), String>>> = OnceCell::new();            // DCA - Deep Common Ancestor entre un par de terminos.


//---------------- Administración de cache.
// Esta función administra las versiones del caché, por ende, permite siempre crear
// una reserva de memoria inicial pero que puede actualizarse según corresponda.
fn set_or_replace_cache<T>(cell: &OnceCell<RwLock<T>>, value: T) {
    if let Some(lock) = cell.get() {
        *lock.write() = value;
    } else {
        let _ = cell.set(RwLock::new(value));
    }
}

//---------------- Estructuras.

/** GAFAnnotation
De este componente se puede comprender que solamente del GAF se obtiene el db_object_id
el go_term y la evidencia asociada a este mismo, siendo las tres columnas claves.
*/
#[pyclass]                      // Se considera una clase compatible con Python.
#[derive(Clone)]                // Permite replicación/copia si es requerido.
pub struct GAFAnnotation {
    #[pyo3(get)]
    pub db_object_id: String,   // Obtención del identificador asociado al gen.
    #[pyo3(get)]
    pub go_term: String,        // GoTerm asociado al gen.
    #[pyo3(get)]
    pub evidence: String,       // Evidencia que respalda la asociación entre ambos.
}

/** TermCounter.
Estructura encargada de establecer la computación de information content (IC) de
cada termino GO asociado.
*/
#[pyclass]
#[derive(Clone)]
pub struct TermCounter {
    #[pyo3(get)]
    pub counts: HashMap<String, usize>,         // Contador de aparición de un térmiono dentro del .gaf
    #[pyo3(get)]
    pub total_by_ns: HashMap<String, usize>,    // Contador de términos únicos por ontología de GO.
    #[pyo3(get)]
    pub ic: HashMap<String, f64>,               // Information Content (IC) de cáda anotación.
}

//---------------- Funciones Implementadas.

/** parse_obo (function)
Función dedicada a la lectura de todos los términos existentes dentro de un archivo .obo para
obtener su información que será guardada en caché, asimismo, se asegura de identificar los
ancestros, computar profundidad-nivel dentro del DAG e identificar los ancestros asociados a
cada uno.
*/
pub fn parse_obo(
    path: &str                                                                                      // Cadena de caracteres del directorio con el .obo.
) 
-> HashMap<String, GOTerm> {
    let contents = fs::read_to_string(path).expect("Can't open OBO file");                          // Carga completa en memoria del archivo
    let chunks = contents.split("[Term]");                                                          // Separación de chunks de líneas según campo [Term]

    let canonical_terms: Vec<GOTerm> = chunks                                                       // Procesamiento paralelo de GoTerms (ignora obsoletos o mal formateados).
        .par_bridge()
        .filter_map(parse_term_chunk)
        .filter(|term| !term.is_obsolete)
        .collect();

    let mut term_map: HashMap<String, GOTerm> =                                                     // Preparación de memoria y configurar uso de hashing rápido.
        HashMap::with_capacity_and_hasher(canonical_terms.len() * 2, Default::default());

    for term in canonical_terms.into_iter() {                                                       // Procesamiento de otros identificadores y términos asociados alternativos
        let mut all_ids = term.alt_ids.clone();                                                     // a un GoTerm de forma en que todos "apunten" a la misma información en HASH.
        all_ids.push(term.id.clone());

        // Este proceso de clnanción lo que hace es que en cada id alternativo
        // almacenado en HASH (procesado en línea anteriores) se le clone
        // los datos del "término principal".
        for id in &all_ids {
            let mut clone = term.clone();
            clone.id = id.clone();
            clone.alt_ids = all_ids.clone();
            term_map.insert(id.clone(), clone);
        }
    }

    // Se tiene que recordar que -
    // Nivel: Distancia mínima desde la "raíz" hacia un nodo.
    // Profundidad: Distancia máxima desde la "raíz" hacia un nodo..
    compute_levels_and_depths(&mut term_map);                                                       // Se Calcula profundidad/especifidad de cada término.

    // Función dedicada a la recolección de ancestro de un GOTerm
    // en particular y que este no haya sido identificado previamente.
    fn collect_ancestors_uncached(
        go_id: &str,                                                                                // Identificador de GoTerms. 
        terms: &HashMap<String, GOTerm>                                                             // Todos los términos mapeados del .obo.
    ) -> HashSet<String> {
        let mut visited = HashSet::default();                                                       // Conjunto de visitados.
        let mut stack = vec![go_id];                                                                // Pila de GoTerms a procesar (inicia con el proporcionado).
        while let Some(current) = stack.pop() {
            if visited.insert(current.to_string()) {                                                // Por cada GoTerm procesado este se señala como visitado.
                if let Some(term) = terms.get(current) {
                    for parent in &term.parents {                                                   // Agrega a la Pila los padres del término procesado (ancestros).
                        stack.push(parent);
                    }
                }
            }
        }
        visited                                                                                     // Se devuelven los ancestros.
    }

    let ancestors_map: HashMap<String, HashSet<String>> = term_map                                  // Se identifican los ancestros a cada GoTerm y se dejan en caché.
        .par_iter()
        .map(|(id, _)| {
            let ancestors = crate::go_ontology::collect_ancestors(id, &term_map)
                .into_iter()
                .map(|s| s.to_string())
                .collect();
            (id.clone(), ancestors)
        })
        .collect();
    set_or_replace_cache(&ANCESTORS_CACHE, ancestors_map);;                                        // Guardado en cache.
    set_or_replace_cache(&DCA_CACHE, HashMap::default());                                          // Se inicializa cache de ancestro más cercano (se calcula por demanda).
    crate::go_semantic::clear_internal_caches();                                                   // 

    term_map                                                                                       // Se retorna mapeo de GoTermns.
}


/** parse_term_chunk (function)
Función encargada de procesar los campos asociados al texto (usual y opcional) que se encuentra
en el archivo .obo existente en la computadora.
*/
fn parse_term_chunk(
    chunk: &str                                                         // Líneas de texto definidas según una clave o sección repetida.
) -> Option<GOTerm> {
    // Definición de la estructura de un término (dentro del archivo).
    let mut term = GOTerm {
        id: String::new(),                                              // Identificador.
        name: String::new(),                                            // Nombre.
        namespace: String::new(),                                       // Ontología BP, MC, CC.
        definition: String::new(),                                      // Definición.
        parents: Vec::new(),                                            // Padres.
        is_obsolete: false,                                             // ¿Es obsoleto?
        alt_ids: Vec::new(),                                            // Ids alternativos.
        replaced_by: None,                                              // Reemplazado por... (si esta obsoleto).
        consider: Vec::new(),                                           // Considerar ¿Qué cosa?.
        synonyms: Vec::new(),                                           // Sinónimos.
        xrefs: Vec::new(),                                              // Referencias asociadas a...
        relationships: Vec::new(),                                      // Relaciones con otros términois.
        comment: None,                                                  // Comentarios.
        children: Vec::new(),                                           // Hijos.
        level: None,                                                    // Nivel.
        depth: None,                                                    // Profundidad/Especificidad.
    };

    let chunk = chunk.split("[Typedef]").next().unwrap_or(chunk);       // Ignorar campos Typedef.

    let lines: Vec<&str> = chunk
        .lines()
        .map(|l| l.trim())                                              // Eliminamos espacios a izquierda y derecha.
        .filter(|l| !l.is_empty())                                      // Quitamos líneas vacías.
        .collect();

    if lines.is_empty() {                                               // Si no se encuentra texto se finaliza la función.                                           
        return None;
    }
    let mut valid = false;

    // Verificación de campos que se obtienen de cada sección existente en Term.
    for line in lines {
        if line.starts_with("id: ") {                                               // Verificación para que sea válido (debe tener campo ID).
            term.id = line["id: ".len()..].to_string();
            valid = true;
        } else if line.starts_with("name: ") {                                      // Verificación del campo de nombre.
            term.name = line["name: ".len()..].to_string();
        } else if line.starts_with("namespace: ") {                                 // Verificación del campo namespace.
            term.namespace = line["namespace: ".len()..].to_string();
        } else if line.starts_with("def: ") {                                       // Verificación de definición.
            term.definition = line["def: ".len()..].to_string();
        } else if line.starts_with("is_a: ") {                                      // Obtención de lisa de padres.
            let parent = line["is_a: ".len()..]
                .split_whitespace()
                .next()
                .unwrap_or("")
                .to_string();
            if !parent.is_empty() {
                term.parents.push(parent);
            }
        } else if line.starts_with("alt_id: ") {                                    // Verificar términos alternativos.
            term.alt_ids.push(line["alt_id: ".len()..].to_string());
        } else if line.starts_with("is_obsolete: true") {                           // Verificar obsolencia.
            term.is_obsolete = true;
        } else if line.starts_with("replaced_by: ") {                               // Obtener id de reemplazo.
            term.replaced_by = Some(line["replaced_by: ".len()..].to_string());
        } else if line.starts_with("consider: ") {                                  // Obtener campo de consideración.
            term.consider.push(line["consider: ".len()..].to_string());
        } else if line.starts_with("synonym: ") {                                   // Obtener campo de sinónimo.
            term.synonyms.push(line["synonym: ".len()..].to_string());
        } else if line.starts_with("xref: ") {                                      // Obtener campo de xref.
            term.xrefs.push(line["xref: ".len()..].to_string());
        } else if line.starts_with("relationship: ") {                              // Obtener campo de relación..
            let rel_def = &line["relationship: ".len()..];
            let mut parts = rel_def.split_whitespace();
            if let (Some(rel), Some(target)) = (parts.next(), parts.next()) {
                term.relationships.push((rel.to_string(), target.to_string()));
            }
        }
    }

    if valid {
        Some(term)                                                                  // Si el termino es válido es añadido, en caso contrario se ignora.
    } else {
        None
    }
}


/** compute_levels_and_depths (function)
Función encargada de cálcular el nivel y profundidad de cada GoTerm asociado.
*/
pub fn compute_levels_and_depths(
    terms: &mut HashMap<String, GOTerm>                                 // Yérminos mepados.
) {
    
    // Paso 1: construir mapa de hijos teniendo en consideración las relaciones
    // is_a para establecer jerarquía padre-hijo.
    let mut child_map: HashMap<String, Vec<String>> = HashMap::default();
    for (id, term) in terms.iter() {
        for parent in &term.parents {
            child_map.entry(parent.clone()).or_default().push(id.clone());
        }
    }

    // Paso 2: inicializar level como una componente que identifica la distancia mínima
    // entre el término raiz y el termino en estudio. Esto como función recursiva interna
    // que selecciona camino según los padres de menor distancia.
    fn init_level(
        term_id: &str,                                                  // Identificador GoTerm.
        terms: &mut HashMap<String, GOTerm>,                            // Mapa de términos.
        visiting: &mut HashSet<String>,                                 // Nodos que se recorren.
    ) -> usize {
        if visiting.contains(term_id) {
            // Ciclo detectado: se evita recursión infinita.
            eprintln!("⚠️ Ciclo detectado en level: {}", term_id);
            return 0;
        }

        // Caso base: Si el nivel ya fue cálculado, este se devuelve
        // se optimiza el cálculo.
        if let Some(level) = terms.get(term_id).and_then(|t| t.level) {
            return level;
        }

        // Seguimiento de nodos que se visitan.
        visiting.insert(term_id.to_string());

        // Se obtienen padres directos.
        let parents = terms
            .get(term_id)
            .map(|t| t.parents.clone())
            .unwrap_or_default();

        // Si no hay se considera el nivel 0 raíz, en caso contrario
        // es llamada recursiva para seguir buscando la raíz.
        let level = if parents.is_empty() {
            0
        } else {
            parents
                .iter()
                .map(|p| init_level(p, terms, visiting))
                .min()
                .unwrap_or(0) + 1
        };

        // Se quita term_id de los que se están procesando para evitar
        // evitar otra llamada innecesaria.
        visiting.remove(term_id);
        if let Some(term) = terms.get_mut(term_id) {
            term.level = Some(level);
        }

        // Se devuelve el nivel.
        level
    }

    // Paso 3: inicializar depth como la distancia más larga a la raíz. Mismo concepto 
    // que punto anterior.
    fn init_depth(
        term_id: &str,
        terms: &mut HashMap<String, GOTerm>,
        visiting: &mut HashSet<String>,
    ) -> usize {
        if visiting.contains(term_id) {
            eprintln!("Ciclo detectado en depth: {}", term_id);
            return 0;
        }

        if let Some(depth) = terms.get(term_id).and_then(|t| t.depth) {
            return depth;
        }

        visiting.insert(term_id.to_string());

        let parents = terms
            .get(term_id)
            .map(|t| t.parents.clone())
            .unwrap_or_default();

        let depth = if parents.is_empty() {
            0
        } else {
            parents
                .iter()
                .map(|p| init_depth(p, terms, visiting))
                .max()
                .unwrap_or(0) + 1
        };

        visiting.remove(term_id);
        if let Some(term) = terms.get_mut(term_id) {
            term.depth = Some(depth);
        }

        depth
    }

    // Paso 4: recorrer todos los términos y calcular level + depth como llamados
    // a las funciones por cada ids.
    let ids: Vec<String> = terms.keys().cloned().collect();
    for id in &ids {
        let mut visiting = HashSet::default();
        init_level(id, terms, &mut visiting);

        let mut visiting = HashSet::default();
        init_depth(id, terms, &mut visiting);
    }

    // Paso 5: rellenar el campo children con los hijos (solo vía is_a).
    for (parent, children) in child_map {
        if let Some(term) = terms.get_mut(&parent) {
            term.children = children;
        }
    }
}


/** download_obo (function)
Obtención de archivo .obo verificando si este existe en la ruta de destino, por defecto,
descarga la versión básica. */
pub fn download_obo() -> Result<String, String> {
    let obo_path = "go-basic.obo";                                      // Buscar en el directorio actual el archivo
    if Path::new(obo_path).exists() {
        return Ok(obo_path.to_string());
    }

    let url = "http://purl.obolibrary.org/obo/go/go-basic.obo";         // En caso de no encontrarlo se descarga.
    println!("Descargando ontología desde: {}", url);
    let response = get(url).map_err(|e| e.to_string())?;

    let content = response.text().map_err(|e| e.to_string())?;
    fs::write(obo_path, content).map_err(|e| e.to_string())?;

    Ok(obo_path.to_string())                                            // Se devuelve como string para procesarlo.
}


/** load_go_terms (function).
En esta función se realiza el llamado al parseo de GoTerms y su guardado en chache
para no necesitar re-ejecuciones de la función.
*/
#[pyfunction]
#[pyo3(signature = (path=None))]                                            // Valor por defecto en Python = None.
pub fn load_go_terms(
    path: Option<String>                                                    // Directorio de archivo .obo.
) -> PyResult<Vec<PyGOTerm>> {
    let path = match path {                                                 // En caso de no encontrar archivo .obo lo descarga.
        Some(p) => p,
        None => download_obo()
            .map_err(|e| pyo3::exceptions::PyIOError::new_err(e))?,
    };
    if !Path::new(&path).exists() {                                         // Si el directorio es incorrecto se informa.
        return Err(pyo3::exceptions::PyIOError::new_err(format!(
            "OBO file not found: {}",
            path
        )));
    }
    let terms_map = parse_obo(&path);                                      // Llamado a función parse_obo.

    set_or_replace_cache(&GO_TERMS_CACHE, terms_map.clone());              // Guardar en la caché global.      
    let terms_vec = terms_map                                              // Devolver lista de PyGOTerm.
        .into_iter()
        .map(|(_, v)| PyGOTerm::from(&v))
        .collect();

    Ok(terms_vec)
}


/** load_gaf (function)
Función encargada de realizar el mapeo de genes con sus respectivos términos
asociados según la información existente en el .gaf
*/ 
#[pyfunction]
pub fn load_gaf(
    path: String                                                            // Directorio en donde se encuantra el archivo .gaf
) -> PyResult<Vec<GAFAnnotation>> {
    // Se verifica acceso al archivo .gaf existente, en caso de que este no pueda ser 
    // accedido, entonces la función termina.
    let file = File::open(&path)
        .map_err(|e| pyo3::exceptions::PyIOError::new_err(e.to_string()))?;
    let reader = BufReader::new(file);

    // Revisar GOTerms obsoletos dentro de lo cargado en caché.
    // Recordar: Debieron haberse cargado en cache los go terms en primer lugar.
    let terms = match crate::go_ontology::get_terms_or_error() {
        Ok(t) => t,
        Err(e) => return Err(e),
    };

    let mut annotations: Vec<GAFAnnotation> = Vec::new();                   // Vector de anotaciones de GAF (parte vacía).
    let mut gene2go: HashMap<String, Vec<String>> = HashMap::default();     // Relación de genes a términos.

    // Lectura linea a linea filtrando aquellas que no empiecen con '!'.
    for line in reader.lines().filter_map(Result::ok).filter(|l| !l.starts_with('!')) {
        let cols: Vec<&str> = line.split('\t').collect();                   // Identificar columnas mediante una separación por tabuladores (son 7).
        if cols.len() < 7 {                                                 // Debe tener como mínimo 7 campos como mínimo.
            continue;
        }

        // Columnas consideradas de cada linea válida.
        let db_object_id = cols[1].to_string();
        let qualifier = cols[3].to_string();
        let mut go_term = cols[4].to_string();
        let evidence = cols[6].to_string();
        let gene = cols[2].to_string();

        // Se ignoran registros de genes con terminos sin evidencia sustentada,
        // asimismo de asociaciones del tipo "El gen NO hace esto".
        // Filter out ND annotations.
        if evidence == "ND" {
            continue;
        }
        // Saltar NOT annotations.
        if qualifier.contains("NOT") {
            continue;
        }


        // Resolver relación entre genes y términos obsoletos.
        if let Some(term) = terms.get(&go_term) {
            if term.is_obsolete {
                if let Some(ref replacement) = term.replaced_by {
                    // 1. Usar el término de reemplazo.
                    go_term = replacement.clone();
                } else if !term.consider.is_empty() {
                    // 2. Si no funcionó el anterior, utilizar el primer término en "Considerar"
                    go_term = term.consider[0].clone();
                } else {
                    // Si no hay término de reemplazo, omitir anotación.
                    continue;
                }
            }
        } else {
            // GO term no encontrado.
            continue;
        }

        // Añadir anotación dentro de la estructura definida.
        annotations.push(GAFAnnotation {
            db_object_id: db_object_id.clone(),
            go_term: go_term.clone(),
            evidence,
        });

        // Hacer actualización de la estructura a cachear.
        gene2go.entry(gene).or_default().push(go_term);
    }

    // Guardar en caché global.
    let _ = GENE2GO_CACHE.set(RwLock::new(gene2go));

    // Se entregan anotaciones como retorno.
    Ok(annotations)
}

/** build_term_counter (function)
Función encargada de ser una encapsuladora de la función interna que se
dedica a cálcular el IC de cada término que se posea utilizando de referencia
las anotaciones obtenidas del .gaf.
*/
#[pyfunction]
pub fn build_term_counter(
    py: Python<'_>,                                                         // Permisis para utilizar estructuras de Python.
    py_annotations: Vec<Py<GAFAnnotation>>,                                 // Vector de anotaciones existente en python.
) -> PyResult<TermCounter> {
    // Obtener los términos GO desde el caché global.
    let terms = get_terms_or_error()?;

    // Convertir las anotaciones de Py<GAFAnnotation> a GAFAnnotation (Rust).
    let annotations: Vec<GAFAnnotation> = py_annotations
        .into_iter()
        .map(|py_ann| py_ann.extract(py))
        .collect::<PyResult<_>>()?;

    // Llamar a la función de conteo interna.
    Ok(_build_term_counter(&annotations, &terms))
}

/** _build_term_counter (function)
Funci+on encargada de calcular el IC de cada GOTerm obtenido.
*/
fn _build_term_counter(
    annotations: &[GAFAnnotation],                                          // Anotaciones del .gaf
    terms: &HashMap<String, GOTerm>,                                        // Términos go de .obo
) -> TermCounter {
    // Se activa paralelismo para la lectura de la cache de ancestro y utilizando una variable
    // de referencia para usarlo como elemento protegido.
    let ancestors_cache_guard = ANCESTORS_CACHE.get().map(|lock| lock.read());
    let ancestors_cache = ancestors_cache_guard.as_deref();

    // Se construye con paralelismo el mapa que relaciona los genes con su término
    // go y los ancestro de este mismo.
    let obj_to_terms: HashMap<&str, HashSet<String>> = annotations
        .par_iter()
        .fold(HashMap::<&str, HashSet<String>>::default, |mut acc, ann| {
            let go_id = ann.go_term.as_str();                                   // Se obtiene go_id de la anotación.
            let entry = acc.entry(ann.db_object_id.as_str()).or_default();      // Se obtiene identificador de gen.

            // Aquí se añade el término mismo con sus ancestro a la relación
            // con el gen obtenido.
            entry.insert(go_id.to_string());
            if let Some(cache) = ancestors_cache {
                if let Some(ancestors) = cache.get(go_id) {
                    entry.extend(ancestors.iter().cloned());
                } else {
                    // Should be rare (unknown GO ID), but keep a correct fallback.
                    entry.extend(collect_ancestors(go_id, terms).into_iter());
                }
            } else {
                entry.extend(collect_ancestors(go_id, terms).into_iter());
            }
            acc
        })
        .reduce(HashMap::<&str, HashSet<String>>::default, |mut acc, map| {     // Aquí se unen los resultados de los núcleos.
            for (k, v) in map {
                acc.entry(k).or_default().extend(v);
            }
            acc
        });

    // Se construye el conteo de genes anotados en cáda GoTerm y términos
    // únicos en cada ontología de GO.
    let (counts, total_by_ns) = obj_to_terms
        .par_iter()
        .fold(
            || (
                HashMap::<String, usize>::default(),
                HashMap::<String, usize>::default(),
            ),
            |(mut counts, mut total_by_ns), (_gene, term_ids)| {
                let mut namespaces_seen: HashSet<&str> = HashSet::default();
                for term_id in term_ids {
                    if let Some(term) = terms.get(term_id.as_str()) {
                        *counts.entry(term_id.clone()).or_insert(0) += 1;
                        namespaces_seen.insert(term.namespace.as_str());
                    }
                }
                for ns in namespaces_seen {
                    *total_by_ns.entry(ns.to_string()).or_insert(0) += 1;
                }
                (counts, total_by_ns)
            },
        )
        .reduce(
            || (
                HashMap::<String, usize>::default(),
                HashMap::<String, usize>::default(),
            ),
            |(mut counts_a, mut total_a), (counts_b, total_b)| {
                for (k, v) in counts_b {
                    *counts_a.entry(k).or_insert(0) += v;
                }
                for (k, v) in total_b {
                    *total_a.entry(k).or_insert(0) += v;
                }
                (counts_a, total_a)
            },
        );

    // Cálculo final del IC: En caso de más información mire
    // la fórmula en la documentación oficial.
    let mut ic: HashMap<String, f64> = HashMap::default();
    for (term_id, count) in &counts {
        if let Some(term) = terms.get(term_id.as_str()) {
            let total = total_by_ns.get(&term.namespace).copied().unwrap_or(1);
            let freq = *count as f64 / total as f64;
            let info_content = if freq > 0.0 { -freq.ln() } else { 0.0 };
            ic.insert(term_id.clone(), info_content);
        }
    }

    TermCounter {
        counts,
        total_by_ns,
        ic,
    }
}

## Código 3 - go_ontology.rs
Archivo encargado de la definición de la estructura utilizada para almacenar la información presente en los archivos .obo.

In [ ]:
// ---------------- Librerias.
use rustc_hash::{FxHashMap as HashMap, FxHashSet as HashSet};       // Invocación de tablas hash.      
use pyo3::prelude::*;                                               // Rust comprende Python (viceversa).
use pyo3::types::PyString;                                          // Se invoca tipo de dato string de Python.
use crate::go_loader::{GO_TERMS_CACHE, GENE2GO_CACHE};              // Se obtiene la cache generada en go_loader (si existe).
use pyo3::exceptions::PyValueError;                                 // Se invoca 'error por valor' de Python.


// ---------------- Estructuras.

/** GoTerm
Estructura enfocada en representar la información de
un término GO.
*/
#[derive(Clone)]
pub struct GOTerm {
    pub id: String,                                     // Identificador del GOTerm.
    pub name: String,                                   // Nombre del término.
    pub namespace: String,                              // Ontología del término.
    pub definition: String,                             // Definición.
    pub parents: Vec<String>,                           // Identificadores padres del término.
    pub children: Vec<String>,                          // Identificadores de los términos hijos.
    pub depth: Option<usize>,                           // Profundidad [Opcional].
    pub level: Option<usize>,                           // Nivel [Opcional].
    pub is_obsolete: bool,                              // ¿Es obsoleto?
    pub alt_ids: Vec<String>,                           // Identificadores de términos alternativos.
    pub replaced_by: Option<String>,                    // En caso de ser obsoleto ¿Hay id's alternativos? [Opcional]
    pub consider: Vec<String>,                          // Otros términos a considerar.
    pub synonyms: Vec<String>,                          // Sinónimos asociados al término.
    pub xrefs: Vec<String>,                             // Pendiente.
    pub relationships: Vec<(String, String)>,           // Relaciones.
    pub comment: Option<String>,                        // Comentarios [Opcional].
}


/** GoTerm
Estructura enfocada en representar la información de
un término GO, pero en su versión expuesta a Python.
*/
#[pyclass]
#[derive(Clone)]
pub struct PyGOTerm {
    #[pyo3(get)] pub id: String,
    #[pyo3(get)] pub name: String,
    #[pyo3(get)] pub namespace: String,
    #[pyo3(get)] pub definition: String,
    #[pyo3(get)] pub parents: Vec<String>,
    #[pyo3(get)] pub children: Vec<String>,
    #[pyo3(get)] pub depth: Option<usize>,
    #[pyo3(get)] pub level: Option<usize>,
    #[pyo3(get)] pub is_obsolete: bool,
    #[pyo3(get)] pub alt_ids: Vec<String>,
    #[pyo3(get)] pub replaced_by: Option<String>,
    #[pyo3(get)] pub consider: Vec<String>,
    #[pyo3(get)] pub synonyms: Vec<String>,
    #[pyo3(get)] pub xrefs: Vec<String>,
    #[pyo3(get)] pub relationships: Vec<(String, String)>,
    #[pyo3(get)] pub comment: Option<String>,
}

// ---------------- Configuraciones PyGoTerm.

// Esta sección de código define como se debe realizar la copia de los
// datos de la estructura GoTerm de Rust a su versión de Python de forma
// en que no existan errores de compatibilidad.
impl From<&GOTerm> for PyGOTerm {
    fn from(term: &GOTerm) -> Self {
        Self {
            id: term.id.clone(),
            name: term.name.clone(),
            namespace: term.namespace.clone(),
            definition: term.definition.clone(),
            parents: term.parents.clone(),
            children: term.children.clone(),
            depth: term.depth,
            level: term.level,
            is_obsolete: term.is_obsolete,
            alt_ids: term.alt_ids.clone(),
            replaced_by: term.replaced_by.clone(),
            consider: term.consider.clone(),
            synonyms: term.synonyms.clone(),
            xrefs: term.xrefs.clone(),
            relationships: term.relationships.clone(),
            comment: term.comment.clone(),
        }
    }
}

// Esta sección de código define el comportamiento de Python a imprimir
// un término GO como un mensaje que considera los elementos definidos
// previamente. Además de automatizar la escritura del nombre del
// objeto.
#[pymethods]
impl PyGOTerm {
    fn __repr__(slf: &Bound<'_, Self>) -> PyResult<String> {
        let class_name: Bound<'_, PyString> = slf.get_type().qualname()?;
        let s = slf.borrow();
        Ok(format!(
            "{} id: {}\nname: {}\nnamespace: {}\ndefinition: {}\nparents: {:?}\nchildren: {:?}\ndepth: {:?}\nlevel: {:?}\nis_obsolete: {}\nalt_ids: {:?}\nreplaced_by: {:?}\nconsider: {:?}\nsynonyms: {:?}\nxrefs: {:?}\nrelationships: {:?}\ncomments: {:?}",
            class_name, s.id, s.name, s.namespace, s.definition, s.parents, s.children, s.depth, s.level,
            s.is_obsolete, s.alt_ids, s.replaced_by, s.consider, s.synonyms, s.xrefs, s.relationships, s.comment
        ))
    }
}

// Esta sección genera un ciclo de regreso de la versión Python a la versión
// de Rust, esto hace que cualquier modificación de una capa afecte a otra.
impl From<PyGOTerm> for GOTerm {
    fn from(py_term: PyGOTerm) -> Self {
        Self {
            id: py_term.id,
            name: py_term.name,
            namespace: py_term.namespace,
            definition: py_term.definition,
            parents: py_term.parents,
            children: py_term.children,
            depth: py_term.depth,
            level: py_term.level,
            is_obsolete: py_term.is_obsolete,
            alt_ids: py_term.alt_ids,
            replaced_by: py_term.replaced_by,
            consider: py_term.consider,
            synonyms: py_term.synonyms,
            xrefs: py_term.xrefs,
            relationships: py_term.relationships,
            comment: py_term.comment,
        }
    }
}

// ---------------- Estructuras.

/** get_terms_or_error (function)
Obtiene de la cache los datos de los GoTerms cargados, en caso
de no existir se arroja error.
*/
pub fn get_terms_or_error<'a>() -> PyResult<parking_lot::RwLockReadGuard<'a, HashMap<String, GOTerm>>> {
    Ok(
        GO_TERMS_CACHE
            .get()
            .ok_or_else(|| pyo3::exceptions::PyRuntimeError::new_err("GO terms not loaded. Call go3.load_go_terms() first."))?
            .read()
    )
}

/** get_gene2go_or_error (function)
Obtiene de la cache los datos del mapeo que relaciona genes con sus términso GO 
asociados, en caso de no existir se arroja error.
*/
pub fn get_gene2go_or_error<'a>() -> PyResult<parking_lot::RwLockReadGuard<'a, HashMap<String, Vec<String>>>> {
    Ok(
        GENE2GO_CACHE
            .get()
            .ok_or_else(|| pyo3::exceptions::PyRuntimeError::new_err("Gene2GO mapping not loaded. Call go3.load_gene2go() first."))?
            .read()
    )
}

/** get_term_by_id (function)
Obtiene un GOTerm según el id proporcionado.
*/
#[pyfunction]
pub fn get_term_by_id(
    go_id: &str                             // Identificador de GoTerms.
) -> PyResult<PyGOTerm> {
    let terms = get_terms_or_error()?;
    match terms.get(go_id) {
        Some(term) => Ok(PyGOTerm::from(term)),
        None => Err(PyValueError::new_err(format!(
            "GO term '{}' not found in ontology",
            go_id
        ))),
    }
}


/** collect_ancestors (function)
Obtiene los ancestros asociados a un GoTerm mediante la relación
is_a.
*/
pub fn collect_ancestors(
    go_id: &str,                            // Identificador de GoTerms.
    terms: &HashMap<String, GOTerm>         // Toda la población de términos.
) -> HashSet<String> {
    // Se intenta revisar si ua se identificaron esos ancestros.
    if let Some(lock) = crate::go_loader::ANCESTORS_CACHE.get() {
        let cache = lock.read();
        if let Some(ancestors) = cache.get(go_id) {
            return ancestors.clone();
        }
    }
    // Se obtienen los padres del GOTerm revisando los padres
    // recursivamente hasta que solo queden los elementos raíz.
    let mut visited = HashSet::default();
    let mut stack = vec![go_id];
    while let Some(current) = stack.pop() {
        if visited.insert(current.to_string()) {
            if let Some(term) = terms.get(current) {
                for parent in &term.parents {
                    stack.push(parent);
                }
            }
        }
    }
    // Ancestros.
    visited
}

/** ancestors (function)
Esto cuenta como un encapsulamiento del código con la finalidad de que
solo el usuario provea el GO id.
*/
#[pyfunction]
pub fn ancestors(go_id: &str) -> PyResult<Vec<String>> {
    let terms = get_terms_or_error()?;
    let visited = collect_ancestors(go_id, &terms);
    Ok(visited.into_iter().collect())
}

/** common_ancestor (function)
Esta función lo que realiza es la intersección de todos los términos
ancestrales entre dos GO ids.
*/
#[pyfunction]
pub fn common_ancestor(
    go_id1: &str,                       // Identificador GOTerm 1
    go_id2: &str                        // Identificador GOTerm 2
) -> PyResult<Vec<String>> {
    let terms = get_terms_or_error()?;
    let set1 = collect_ancestors(go_id1, &terms);
    let set2 = collect_ancestors(go_id2, &terms);
    let mut common: Vec<String> = set1.intersection(&set2).map(|s| (*s).to_string()).collect();
    common.sort_unstable();
    Ok(common)
}

/** deepest_common_ancestor (function)
Obtiene el ancestro más profundo entre 2 GOTerms.
*/
#[pyfunction]
pub fn deepest_common_ancestor(go_id1: &str, go_id2: &str) -> PyResult<Option<String>> {
    let terms = get_terms_or_error()?;                                      // Se buscan términos dentro de la cache existente.

    if !terms.contains_key(go_id1) {                                        // Verificación de que los terminos existan dentro del mapeo de términos.
        return Err(PyValueError::new_err(format!(
            "GO term '{}' not found in ontology",
            go_id1
        )));
    }
    if !terms.contains_key(go_id2) {
        return Err(PyValueError::new_err(format!(
            "GO term '{}' not found in ontology",
            go_id2
        )));
    }

    // Se establece el par ordenado de ambos ids, ya que esto estandariza
    // los pares para consulta en cache, como estandarizar que el valor
    // es independiente del orden.
    let (id_a, id_b) = if go_id1 <= go_id2 {
        (go_id1, go_id2)
    } else {
        (go_id2, go_id1)
    };

    // Verificar si en caché existió un precalculo del DCA previamente.
    if let Some(lock) = crate::go_loader::DCA_CACHE.get() {
        let cache = lock.write();
        if let Some(result) = cache.get(&(id_a.to_string(), id_b.to_string())) {
            return Ok(Some(result.clone()));
        }
    }

    // En caso de no encontrar ancestro en caché.
    let set1 = collect_ancestors(id_a, &terms);                             // Ancestros del id_a.
    let set2 = collect_ancestors(id_b, &terms);                             // Ancestros del id_b.
    let mut best = None;
    let mut max_depth = 0;
    for term_id in set1.intersection(&set2) {                               // Intersección entre ambas listas de ancestros.
        if let Some(term) = terms.get(term_id) {                            // Se verifican campos depth para obtener el DCA.
            if let Some(depth) = term.depth {
                if depth >= max_depth {
                    max_depth = depth;
                    best = Some(term_id.to_string());
                }
            }
        }
    }

    // Guardar reconocimiento de ancestro en caché.
    if let Some(lock) = crate::go_loader::DCA_CACHE.get() {
        let mut cache = lock.write();
        if let Some(ref dca) = best {
            cache.insert((id_a.to_string(), id_b.to_string()), dca.clone());
        }
    }

    Ok(best)
}

## Conjunto de códigos - go_semantic
En versiones iniciales de la librería el archivo semantic.rs proporcionaba las funciones de cálculo de similitud, no obstante, tras una actualización se proporcionó un conjunto de archivos que realizan esta actividad.

In [ ]:
// mod.rs: Archivo encargado de proporcionar todas las funcionalidades de la carpeta de forma
// que estas solo se consideren como un único conjunto invocable.
mod embedding;
mod gene;
mod similarity;
mod termset;

// Especificación de funciones asociadas a go_semantic.
pub use embedding::{plot_embedding, plot_tsne_genes, plot_umap_genes, tsne_genes, umap_genes};
pub use gene::{compare_gene_pairs_batch, compare_genes, gene_distance_matrix};
pub use similarity::{batch_similarity, semantic_similarity, set_num_threads, term_ic};
pub use termset::termset_similarity;

// Se establece función de limieza de caché.
pub(crate) fn clear_internal_caches() {
    similarity::clear_internal_caches();
}

In [ ]:
//------------------------------- Liberías.
use pyo3::exceptions::PyValueError;                                                     // Invocación de errores ValueError para usuario.
use pyo3::prelude::*;                                                                   // Usar Rust en Python.
use rayon::prelude::*;                                                                  // Rust comprende Python.
use rayon::ThreadPoolBuilder;                                                           // Administración de hilos.
use rustc_hash::{FxHashMap as HashMap, FxHashSet as HashSet};                           // Implementación de tablas hash.

use crate::go_loader::{GO_TERMS_CACHE, TermCounter};                                    // Utilizar estructuras cache de go_loader.
// Utilizar elementos asociados a código de go_ontology.
use crate::go_ontology::{collect_ancestors, deepest_common_ancestor, get_term_by_id, GOTerm};
use dashmap::DashMap;                                                                   // Tablas hash que se les aplica concurrencia.
use std::sync::Arc;                                                                     // Sincronización de hilos.


//------------------------------- Estructuras.
/*Encargada de leer la contribución semántica.*/
static SEMANTIC_CONTRIB_CACHE: once_cell::sync::Lazy<
    DashMap<String, Arc<HashMap<String, f64>>>,
> = once_cell::sync::Lazy::new(DashMap::new);

pub(crate) fn clear_internal_caches() {
    SEMANTIC_CONTRIB_CACHE.clear();
}

//------------------------------- Funciones.

/** set_num_threads (function)
Configuración de hilos que Rust puede utilizar de la máquina.
*/
#[pyfunction]
pub fn set_num_threads(n_threads: usize) -> PyResult<()> {
    let builder = if n_threads == 0 {
        ThreadPoolBuilder::new()                            // En caso de colocar 0 o nada se utilizan todos.
    } else {
        ThreadPoolBuilder::new().num_threads(n_threads)     // Si se especifica se establece el número de hilos requeridos.
    };

    //Se revisa si esta inicialización fue hecha previamente por
    //el proceso para evitar una creación indiscriminada de hilos.
    match builder.build_global() {
        Ok(()) => Ok(()),
        Err(e) => {
            let msg = e.to_string();
            if msg.contains("already been initialized") {
                Ok(())
            } else {
                Err(pyo3::exceptions::PyRuntimeError::new_err(msg))
            }
        }
    }
}


/** term_ic (function)
Se obtienen los information content (IC) ya precomputados de 
go_loader.
*/
#[pyfunction]
#[pyo3(text_signature = "(go_id, counter)")]
pub fn term_ic(go_id: &str, counter: &TermCounter) -> f64 {
    *counter.ic.get(go_id).unwrap_or(&0.0)
}

/*
Esta sección de código administra la invocación de algún método
de similitud según el texto que ingrese el usuario en los parámetros
al utilizar la función en Python.
*/
#[derive(Debug, Clone, Copy)]
pub(crate) enum SimilarityMethod {
    Resnik,
    Lin,
    JC,
    SimRel,
    ICCoef,
    GraphIC,
    Wang,
    TopoICSim,
}

/** SimilarityMethod (Interfaz de funciones)
En este código se invocan todos los cálculos de métodos de similitud solicitados por el
usuario, en resumidos término este código obedece todas las fórmulas planteadas en la 
documentación provista por el autor, por ende, se recomienda guiarse por esa información.
*/
impl SimilarityMethod {
    pub(crate) fn from_str(name: &str) -> Option<Self> {
        match name.to_ascii_lowercase().as_str() {
            "resnik" => Some(SimilarityMethod::Resnik),
            "lin" => Some(SimilarityMethod::Lin),
            "jc" => Some(SimilarityMethod::JC),
            "simrel" => Some(SimilarityMethod::SimRel),
            "iccoef" => Some(SimilarityMethod::ICCoef),
            "graphic" => Some(SimilarityMethod::GraphIC),
            "wang" => Some(SimilarityMethod::Wang),
            "topoicsim" => Some(SimilarityMethod::TopoICSim),
            _ => None,
        }
    }

    pub(crate) fn compute(&self, id1: &str, id2: &str, counter: &TermCounter) -> f64 {
        match self {
            SimilarityMethod::Resnik => {
                let dca = match deepest_common_ancestor(id1, id2).ok().flatten() {
                    Some(dca) => dca,
                    None => return 0.0,
                };
                *counter.ic.get(&dca).unwrap_or(&0.0)
            }
            SimilarityMethod::Lin => {
                let dca = match deepest_common_ancestor(id1, id2).ok().flatten() {
                    Some(dca) => dca,
                    None => return 0.0,
                };
                if id1 == id2 {
                    return 1.0
                }
                let resnik = *counter.ic.get(&dca).unwrap_or(&0.0);
                if resnik == 0.0 {
                    return 0.0;
                }
                let ic1 = *counter.ic.get(id1).unwrap_or(&0.0);
                let ic2 = *counter.ic.get(id2).unwrap_or(&0.0);
                if ic1 == 0.0 || ic2 == 0.0 {
                    return 0.0;
                }
                2.0 * resnik / (ic1 + ic2)
            }
            SimilarityMethod::JC => {
                let (t1, t2) = match (get_term_by_id(id1).ok(), get_term_by_id(id2).ok()) {
                    (Some(t1), Some(t2)) => (t1, t2),
                    _ => return 0.0,
                };
            
                if t1.namespace != t2.namespace {
                    return 0.0;
                }
            
                let ic1 = term_ic(id1, counter);
                let ic2 = term_ic(id2, counter);
            
                let dca_ic = match deepest_common_ancestor(id1, id2).ok().flatten() {
                    Some(dca) => term_ic(&dca, counter),
                    None => return 0.0,
                };
            
                let distance = ic1 + ic2 - 2.0 * dca_ic;
                if distance <= 0.0 {
                    return f64::INFINITY;  // Máxima similitud
                }
                if distance.is_infinite() {
                    0.0
                } else {
                    1.0 / (1.0 + distance)
                }
            }
            SimilarityMethod::SimRel => {
                let (t1, t2) = match (get_term_by_id(id1).ok(), get_term_by_id(id2).ok()) {
                    (Some(t1), Some(t2)) => (t1, t2),
                    _ => return 0.0,
                };
            
                if t1.namespace != t2.namespace {
                    return 0.0;
                }
            
                let ic1 = term_ic(id1, counter);
                let ic2 = term_ic(id2, counter);
            
                if ic1 == 0.0 || ic2 == 0.0 {
                    return 0.0;
                }
            
                let dca_ic = match deepest_common_ancestor(id1, id2).ok().flatten() {
                    Some(dca) => term_ic(&dca, counter),
                    None => return 0.0,
                };
            
                if dca_ic == 0.0 {
                    return 0.0;
                }
            
                let lin = (2.0 * dca_ic) / (ic1 + ic2);
                lin * (1.0 - (-dca_ic).exp())
            }
            SimilarityMethod::ICCoef => {
                let (t1, t2) = match (get_term_by_id(id1).ok(), get_term_by_id(id2).ok()) {
                    (Some(t1), Some(t2)) => (t1, t2),
                    _ => return 0.0,
                };
            
                if t1.namespace != t2.namespace {
                    return 0.0;
                }
            
                let ic1 = term_ic(id1, counter);
                let ic2 = term_ic(id2, counter);
            
                if ic1 == 0.0 || ic2 == 0.0 {
                    return 0.0;
                }
            
                let dca_ic = match deepest_common_ancestor(id1, id2).ok().flatten() {
                    Some(dca) => term_ic(&dca, counter),
                    None => return 0.0,
                };
            
                dca_ic / ic1.min(ic2)
            }
            SimilarityMethod::GraphIC => {
                let (t1, t2) = match (get_term_by_id(id1).ok(), get_term_by_id(id2).ok()) {
                    (Some(t1), Some(t2)) => (t1, t2),
                    _ => return 0.0,
                };
            
                if t1.namespace != t2.namespace {
                    return 0.0;
                }
            
                let depth1 = t1.depth.unwrap_or(0);
                let depth2 = t2.depth.unwrap_or(0);
                let max_depth = (depth1.max(depth2) + 1) as f64;
            
                let dca_ic = match deepest_common_ancestor(id1, id2).ok().flatten() {
                    Some(dca) => term_ic(&dca, counter),
                    None => return 0.0,
                };
            
                dca_ic / max_depth
            }
            SimilarityMethod::Wang => {
                let terms = match GO_TERMS_CACHE.get() {
                    Some(lock) => lock.read(),
                    None => return 0.0,
                };
            
                let terms = &*terms;
            
                let t1 = match terms.get(id1) {
                    Some(t) => t,
                    None => return 0.0,
                };
                let t2 = match terms.get(id2) {
                    Some(t) => t,
                    None => return 0.0,
                };
            
                if t1.namespace != t2.namespace {
                    return 0.0;
                }
            
                let sv_a = semantic_contributions(id1, terms);
                let sv_b = semantic_contributions(id2, terms);

                let sum_a: f64 = sv_a.values().sum();
                let sum_b: f64 = sv_b.values().sum();

                let (small, large) = if sv_a.len() <= sv_b.len() {
                    (&*sv_a, &*sv_b)
                } else {
                    (&*sv_b, &*sv_a)
                };

                let mut numerator = 0.0;
                for (key, w1) in small.iter() {
                    if let Some(w2) = large.get(key) {
                        numerator += (*w1).min(*w2);
                    }
                }
            
                if sum_a + sum_b == 0.0 {
                    0.0
                } else {
                    numerator / ((sum_a + sum_b) / 2.0)
                }
            }
            SimilarityMethod::TopoICSim => {
                // Get terms and check namespace
                let terms = match GO_TERMS_CACHE.get() {
                    Some(lock) => lock.read(),
                    None => return 0.0,
                };
                let t1 = match terms.get(id1) {
                    Some(t) => t,
                    None => return 0.0,
                };
                let t2 = match terms.get(id2) {
                    Some(t) => t,
                    None => return 0.0,
                };
                if t1.namespace != t2.namespace {
                    return 0.0;
                }
                // Disjunctive common ancestors
                let dca_set = disjunctive_common_ancestors(id1, id2, &terms);
                if dca_set.is_empty() {
                    return 0.0;
                }
                // Find all roots
                let roots = find_roots(&terms);
                if roots.is_empty() {
                    return 0.0;
                }
                let mut min_d = f64::INFINITY;
                for x in dca_set {
                    // Weighted shortest path from t1 to x and t2 to x
                    let wsp1 = weighted_shortest_path_iic(id1, &x, &terms, counter);
                    let wsp2 = weighted_shortest_path_iic(id2, &x, &terms, counter);
                    if wsp1.is_none() || wsp2.is_none() {
                        continue;
                    }
                    let wsp = wsp1.unwrap() + wsp2.unwrap();
                    // Weighted longest path from x to any root (take the max over all roots)
                    let mut max_wlp = None;
                    for root in &roots {
                        if let Some(wlp) = weighted_longest_path_iic(&x, root, &terms, counter) {
                            max_wlp = Some(max_wlp.map_or(wlp, |m: f64| m.max(wlp)));
                        }
                    }
                    let wlp = match max_wlp {
                        Some(val) if val > 0.0 => val,
                        _ => continue,
                    };
                    let d = wsp / wlp;
                    if d < min_d {
                        min_d = d;
                    }
                }
                if !min_d.is_finite() {
                    return 0.0;
                }
                // Similarity formula: 1 - (arctan(D) / (pi/2))
                let sim = 1.0 - (min_d.atan() / (std::f64::consts::FRAC_PI_2));
                if sim.is_finite() && sim > 0.0 { sim } else { 0.0 }
            }
        }
    }
}

/** semantic_contributions (function)
Realiza el cálculo de la contribución de todos los ancestros asociados
a un GOTerm según los valores establecidos en literatura. Este componente
solo se utiliza para el índice de Wang.
*/
fn semantic_contributions(
    go_id: &str,                                                // Identificador de GoTerm.
    terms: &HashMap<String, GOTerm>,                            // Conjunto de términos GO.
) -> Arc<HashMap<String, f64>> {
    // Verifica si el cálculo ya esta en caché.
    if let Some(cached) = SEMANTIC_CONTRIB_CACHE.get(go_id) {
        return Arc::clone(cached.value());
    }

    // Inicialización de variables.
    let mut contributions = HashMap::default();                 // Contribuyentes.
    let mut to_visit: Vec<(&str, f64)> = vec![(go_id, 1.0)];    // Primer término de la lista es el mismo de entrada.

    while let Some((current_id, weight)) = to_visit.pop() {
        // Si la contribución es muy pequeña, se omite.
        if weight < 1e-6 || contributions.contains_key(current_id) {
            continue;
        }

        contributions.insert(current_id.to_string(), weight);

        if let Some(term) = terms.get(current_id) {
            // is_a → 0.8 si es padre.
            for parent in &term.parents {
                to_visit.push((parent.as_str(), weight * 0.8));
            }
            // part_of → 0.6 si es relacionado.
            for (rel_type, target) in &term.relationships {
                if rel_type == "part_of" {
                    to_visit.push((target.as_str(), weight * 0.6));
                }
            }
        }
    }

    // Contribuciones se guarda en caché.
    let contributions = Arc::new(contributions);
    SEMANTIC_CONTRIB_CACHE.insert(go_id.to_string(), Arc::clone(&contributions));
    contributions
}


//------------------------------- Funciones TopoICSim.

/** iic (function)
Calcula el Inverse Information Content (IIC) para un término.
*/
fn iic(go_id: &str, counter: &TermCounter) -> f64 {
    let ic = *counter.ic.get(go_id).unwrap_or(&0.0);
    if ic > 0.0 {
        1.0 / ic
    } else {
        // Si el IC es 0, el IIC se trata como un valor extremadamente alto.
        1e12
    }
}

/** weighted_shortest_path_iic (function)
Se calcula el camino más corto según IIC entre dos GOTerms.
*/
fn weighted_shortest_path_iic(
    source: &str,                                   // GoTerm de origen.
    target: &str,                                   // Goterm objetivo.
    terms: &HashMap<String, GOTerm>,                // Conjunto de términos.
    counter: &TermCounter,                          // TermCounter.
) -> Option<f64> {
    // Usamos estructuras estándar: un heap (cola de prioridad) y un HashMap local.
    use std::collections::{BinaryHeap, HashMap};
    use std::cmp::Ordering;

    // Estado que guardamos en el heap:
    // - `cost`: costo acumulado para llegar a este nodo
    // - `node`: índice numérico del nodo (más eficiente que usar String)
    #[derive(Copy, Clone, PartialEq)]
    struct State {
        cost: f64,
        node: usize,
    }

    // Necesario para que `State` pueda ir dentro de BinaryHeap.
    impl Eq for State {}

    // `BinaryHeap` en Rust es un *max-heap* por defecto (saca el mayor primero).
    // Para que se comporte como *min-heap* (sacar el menor costo primero),
    // invertimos la comparación (`other` vs `self`).
    impl Ord for State {
        fn cmp(&self, other: &Self) -> Ordering {
            // Orden invertido → el menor `cost` queda “más alto” en prioridad.
            other.cost.partial_cmp(&self.cost).unwrap_or(Ordering::Equal)
        }
    }

    impl PartialOrd for State {
        fn partial_cmp(&self, other: &Self) -> Option<Ordering> {
            Some(self.cmp(other))
        }
    }


    // -------------------------------------------------------------------------
    // 1) Construir un mapeo entre IDs (String) y índices (usize)
    //
    // ¿Por qué?
    // - `dist` será un Vec<f64> (mucho más rápido que HashMap en cada actualización)
    // - Para usar Vec necesitamos índices numéricos.
    // -------------------------------------------------------------------------
    let mut id2idx = HashMap::new();            // GO_ID -> índice
    let mut idx2id = Vec::new();                // índice -> GO_ID

    for (i, id) in terms.keys().enumerate() {
        id2idx.insert(id.as_str(), i);
        idx2id.push(id.as_str());
    }

    // Verificar que `source` y `target` existen en el grafo.
    let src = match id2idx.get(source) {
        Some(&i) => i,
        None => return None,
    };
    let tgt = match id2idx.get(target) {
        Some(&i) => i,
        None => return None,
    };

    // -------------------------------------------------------------------------
    // 2) Inicializar estructuras de Dijkstra
    //
    // `dist[i]` = mejor (menor) costo conocido para llegar al nodo i.
    // Partimos con infinito para todos, excepto `source`.
    // -------------------------------------------------------------------------
    let mut dist = vec![f64::INFINITY; idx2id.len()];

    // Costo inicial:
    // - En esta implementación, el costo incluye el IIC del nodo inicial.
    dist[src] = iic(source, counter);

    // Heap (cola de prioridad) con el nodo inicial.
    let mut heap = BinaryHeap::new();
    heap.push(State { cost: dist[src], node: src });

    // -------------------------------------------------------------------------
    // 3) Bucle principal de Dijkstra
    //
    // Siempre tomamos el nodo pendiente con menor costo acumulado.
    // -------------------------------------------------------------------------
    while let Some(State { cost, node }) = heap.pop() {
        let node_id = idx2id[node];

        // Si ya llegamos al destino, el costo actual es el menor posible.
        if node == tgt {
            return Some(cost);
        }

        // Si este estado no coincide con la mejor distancia conocida, lo ignoramos.
        // (Esto ocurre porque en el heap pueden quedar entradas antiguas)
        if cost > dist[node] {
            continue;
        }

        // ---------------------------------------------------------------------
        // 4) Explorar vecinos (en GO: subir hacia los padres)
        //
        // Aquí el “vecino” de un nodo son sus padres.
        // Cada paso añade el peso IIC del padre al costo acumulado.
        // ---------------------------------------------------------------------
        if let Some(term) = terms.get(node_id) {
            for parent in &term.parents {
                if let Some(&parent_idx) = id2idx.get(parent.as_str()) {
                    // Nuevo costo candidato si vamos desde `node` hacia `parent`.
                    let next = cost + iic(parent, counter);

                    // Si encontramos un camino más barato hacia el padre, lo guardamos.
                    if next < dist[parent_idx] {
                        dist[parent_idx] = next;
                        heap.push(State { cost: next, node: parent_idx });
                    }
                }
            }
        }
    }

    // Si se vació el heap y nunca llegamos a `target`, no hay conexión.
    None

}

/// Compute the weighted longest path (sum of IICs) from source to target (ancestor) in the GO DAG.
///
/// Arguments
/// ---------
/// source : str
///   Source GO term ID.
/// target : str
///   Target GO term ID (ancestor).
/// terms : dict
///   Map of GO terms.
/// counter : TermCounter
///   Precomputed term counter with IC values.
///
/// Returns
/// -------
/// Option<float>
///   Maximum sum of IICs along any path from source to target, or None if not connected.

/** weighted_longest_path_iic (function)
Se calcula el camino más largo según IIC entre dos GOTerms.
*/
fn weighted_longest_path_iic(
    source: &str,                                   // GoTerm de origen.
    target: &str,                                   // Goterm objetivo.
    terms: &HashMap<String, GOTerm>,                // Conjunto de términos.
    counter: &TermCounter,                          // TermCounter.
) -> Option<f64> {
    // Usamos HashMap estándar solo para la memoización local.
    use std::collections::HashMap as StdHashMap;

    // -------------------------------------------------------------------------
    // DFS con memoización:
    // dfs(node) retorna la *máxima* suma de IIC desde `node` hasta `target`.
    // -------------------------------------------------------------------------
    fn dfs(
        node: &str,
        target: &str,
        terms: &HashMap<String, GOTerm>,
        counter: &TermCounter,
        memo: &mut StdHashMap<String, Option<f64>>,
    ) -> Option<f64> {
        // Caso base:
        // - Si ya estamos en el destino, el mejor camino es “quedarse aquí”.
        // - Incluimos el IIC del propio `target`.
        if node == target {
            return Some(iic(node, counter));
        }

        // Si ya calculamos la respuesta para `node`, la reutilizamos.
        // Esto evita recomputar el mismo subgrafo muchas veces.
        if let Some(&val) = memo.get(node) {
            return val;
        }

        // Aquí iremos guardando el mejor (máximo) camino encontrado.
        let mut max_path = None;

        // Si el nodo existe, probamos cada padre como “siguiente paso”.
        if let Some(term) = terms.get(node) {
            // Preguntamos: “¿Existe un camino desde el padre hasta target?”
            for parent in &term.parents {
                if let Some(sub) = dfs(parent, target, terms, counter, memo) {
                    // Si existe, el camino total desde `node` es:
                    // IIC(node) + (mejor camino desde parent hasta target)
                    let total = iic(node, counter) + sub;

                    // Nos quedamos con el máximo entre los caminos posibles.
                    max_path = Some(max_path.map_or(total, |m: f64| m.max(total)));
                }
            }
        }
        // Guardamos el resultado para `node` en memo (aunque sea None).
        memo.insert(node.to_string(), max_path);
        max_path
    }

    // Memo vacía al inicio.
    let mut memo = StdHashMap::new();

    // Ejecutamos la DFS desde `source`.
    dfs(source, target, terms, counter, &mut memo)
}

/** disjunctive_common_ancestors (function)
Obtener los ancestros comunes disyuntivos entre un par de GOTerms.
*/
fn disjunctive_common_ancestors(
    id1: &str,
    id2: &str,
    terms: &HashMap<String, GOTerm>,
) -> Vec<String> {
    use std::collections::HashSet;
    // Se obtiene todos los ancestros comunes entre los dos términos de entrada.
    let ancestors1 = collect_ancestors(id1, terms);
    let ancestors2 = collect_ancestors(id2, terms);
    let common: HashSet<_> = ancestors1.intersection(&ancestors2).cloned().collect();

    // Aquí se comprueba la consición de disyuntivo, es decir, que el término es
    // un ancestro de ambos GOTerms pero sus hijos no son ancestros compartidos.
    let mut dca = Vec::new();
    for x in &common {
        let is_disjunctive = terms.get(x).map_or(false, |term| {
            term.children.iter().all(|c| !common.contains(c))
        });
        if is_disjunctive {
            dca.push(x.clone());
        }
    }
    dca
}


/** find_roots (function)
Obtener los términos raíz (sin padres) del conjunto total de GoTerms.
*/
fn find_roots(terms: &HashMap<String, GOTerm>) -> Vec<String> {
    terms.iter()
        .filter(|(_, term)| term.parents.is_empty())
        .map(|(id, _)| id.clone())
        .collect()
}


/** semantic_similarity (function)
Función capsula para el cálculo de similitud entre 2 GOTerms.
*/
#[pyfunction]
pub fn semantic_similarity(
    id1: &str,
    id2: &str,
    method: &str,
    counter: &TermCounter,
) -> PyResult<f64> {

    let method_enum = SimilarityMethod::from_str(method)
        .ok_or_else(|| PyValueError::new_err(format!("Unknown similarity method: {}", method)))?;

    Ok(method_enum.compute(id1, id2, counter))
}

/** semantic_similarity (function)
Función capsula para el cálculo de similitud entre 2 listas de GOTerms.
*/
#[pyfunction]
pub fn batch_similarity(
    list1: Vec<String>,
    list2: Vec<String>,
    method: &str,
    counter: &TermCounter,
) -> PyResult<Vec<f64>> {
    // Se comprueba que ambas listas tengan el mismo tamaño.
    if list1.len() != list2.len() {
        return Err(PyValueError::new_err("Both lists must be the same length"));
    }

    // Se establece método de similitud según lo indicado en los parametros
    // si no se obtiene respuesta, entonces se señala que el método escrito no es
    // valido.
    let method_enum = SimilarityMethod::from_str(method)
        .ok_or_else(|| PyValueError::new_err(format!("Unknown similarity method: {}", method)))?;

    
    // ---------------------------------------------------------------------
    // "Interning" de términos: asignar a cada GO ID un índice numérico.
    //
    // ¿Por qué?
    // - Para evitar copiar Strings y usarlas como keys una y otra vez.
    // - Trabajar con `usize` (índices) es más rápido y barato en memoria.
    //
    // term_to_idx: &str (GO ID) -> índice
    // idx_to_term: índice -> &str (GO ID)
    //
    // Nota importante:
    // - Estos &str apuntan a las Strings dentro de list1/list2.
    // - Esto es válido porque list1/list2 viven durante toda la función.
    // ---------------------------------------------------------------------
    let mut term_to_idx: HashMap<&str, usize> = HashMap::default();
    let mut idx_to_term: Vec<&str> = Vec::new();
    for s in list1.iter().chain(list2.iter()) {
        let key = s.as_str();
        if term_to_idx.get(key).is_none() {
            let idx = idx_to_term.len();
            term_to_idx.insert(key, idx);
            idx_to_term.push(key);
        }
    }


    // ---------------------------------------------------------------------
    // Recolectar pares únicos (sin orden) para no recalcular similitudes.
    //
    // Si aparecen pares repetidos, por ejemplo:
    //   (A, B) muchas veces
    // solo calculamos una vez su similitud.
    //
    // "sin orden" significa:
    // - (A,B) y (B,A) se consideran el mismo par
    // - se normaliza usando (min(i,j), max(i,j))
    //
    // unique_pairs: conjunto de pares únicos ya normalizados (i <= j)
    // pair_indices: lista de pares normalizados en el mismo orden del input,
    //               para luego reconstruir el resultado por posición.
    // ---------------------------------------------------------------------
    let mut unique_pairs: HashSet<(usize, usize)> = HashSet::default();
    let mut pair_indices: Vec<(usize, usize)> = Vec::with_capacity(list1.len());
    for (a, b) in list1.iter().zip(list2.iter()) {
        let i = *term_to_idx
            .get(a.as_str())
            .expect("intern table should contain all terms in list1");
        let j = *term_to_idx
            .get(b.as_str())
            .expect("intern table should contain all terms in list2");
        let key = if i <= j { (i, j) } else { (j, i) };
        unique_pairs.insert(key);
        pair_indices.push(key);
    }

    // Computación de las similitudes entre pares de forma paralela.
    let sim_map: HashMap<(usize, usize), f64> = unique_pairs
        .par_iter()
        .map(|(i, j)| {
            let a = idx_to_term[*i];
            let b = idx_to_term[*j];
            let sim = method_enum.compute(a, b, counter);
            ((*i, *j), sim)
        })
        .collect();
    let result: Vec<f64> = pair_indices
        .par_iter()
        .map(|(i, j)| *sim_map.get(&(*i, *j)).unwrap_or(&0.0))
        .collect();

    Ok(result)
}


In [ ]:
//------------------------------- Liberías.
use pyo3::exceptions::PyValueError;                                                     // Invocación de errores ValueError para usuario.
use pyo3::prelude::*;                                                                   // Usar Rust en Python.
use rayon::prelude::*;                                                                  // Rust comprende Python.
use rustc_hash::{FxHashMap as HashMap, FxHashSet as HashSet};                           // Implementación de tablas hash.

// Se obtienen componentes externas de otros archivos
use crate::go_loader::TermCounter;                                                      // Term Counter.
use crate::go_ontology::{collect_ancestors, get_terms_or_error, GOTerm};                // Manejo del mapeo de GO.
use super::similarity::{term_ic, SimilarityMethod};                                     // Uso de IC y métodos de similitud.

/** termset_similarity_internal_with_method (function)
Esta función esta encargada de la inicialización 
*/
pub(crate) fn termset_similarity_internal_with_method(
    terms1: &[String],
    terms2: &[String],
    sim_fn: Option<SimilarityMethod>,
    groupwise: &str,
    counter: &TermCounter,
    ontology_terms: &HashMap<String, GOTerm>,
) -> PyResult<f64> {
    if terms1.is_empty() || terms2.is_empty() {
        return Ok(0.0);
    }

    if groupwise == "simgic" {
        // Collect all ancestors for each set
        let mut ancestors1: HashSet<String> = HashSet::default();
        for t in terms1 {
            let ancs = collect_ancestors(t, ontology_terms);
            for a in ancs {
                ancestors1.insert(a);
            }
        }
        let mut ancestors2: HashSet<String> = HashSet::default();
        for t in terms2 {
            let ancs = collect_ancestors(t, ontology_terms);
            for a in ancs {
                ancestors2.insert(a);
            }
        }
        
        // Compute Jaccard Index weighted by IC
        let mut intersection_ic = 0.0;
        let mut union_ic = 0.0;
        
        let all_ancestors: HashSet<&String> = ancestors1.union(&ancestors2).collect();

        for term in all_ancestors {
            let ic = term_ic(term, counter);
            let in_1 = ancestors1.contains(term);
            let in_2 = ancestors2.contains(term);
            
            if in_1 && in_2 {
                intersection_ic += ic;
            }
            if in_1 || in_2 {
                union_ic += ic;
            }
        }
        
        if union_ic == 0.0 {
            return Ok(0.0);
        }
        return Ok(intersection_ic / union_ic);
    }
    
    // For other methods, we need the pairwise similarity function
    let sim_fn = sim_fn.ok_or_else(|| {
        PyValueError::new_err("similarity argument is required for this groupwise method")
    })?;

    // Avoid nested rayon parallelism when this function is called from an already-parallel
    // context (e.g. gene_distance_matrix or compare_gene_pairs_batch). In those cases, the
    // outer loop should control parallelism, and we compute the termset score serially.
    let use_parallel = rayon::current_thread_index().is_none();

    match groupwise {
        "max" => {
            let max_val = if use_parallel {
                terms1
                    .par_iter()
                    .map(|id1| {
                        terms2
                            .iter()
                            .map(|id2| sim_fn.compute(id1, id2, counter))
                            .fold(0.0, f64::max)
                    })
                    .reduce(|| 0.0, f64::max)
            } else {
                let mut max_val: f64 = 0.0;
                for id1 in terms1 {
                    for id2 in terms2 {
                        max_val = max_val.max(sim_fn.compute(id1, id2, counter));
                    }
                }
                max_val
            };
            Ok(max_val)
        }
        "bma" => {
            let total = (terms1.len() + terms2.len()) as f64;
            if total == 0.0 {
                return Ok(0.0);
            }

            // Compute row maxima for terms1 and column maxima for terms2 in a single pass
            // over the cartesian product. This avoids doing 2× work for symmetric similarities.
            if use_parallel {
                let (sum_row_max, col_max) = terms1
                    .par_iter()
                    .fold(
                        || (0.0_f64, vec![0.0_f64; terms2.len()]),
                        |(sum, mut col_max), id1| {
                            let mut row_max: f64 = 0.0;
                            for (j, id2) in terms2.iter().enumerate() {
                                let s = sim_fn.compute(id1, id2, counter);
                                if s > row_max {
                                    row_max = s;
                                }
                                if s > col_max[j] {
                                    col_max[j] = s;
                                }
                            }
                            (sum + row_max, col_max)
                        },
                    )
                    .reduce(
                        || (0.0_f64, vec![0.0_f64; terms2.len()]),
                        |(sum_a, mut col_a), (sum_b, col_b)| {
                            for (a, b) in col_a.iter_mut().zip(col_b.iter()) {
                                if *b > *a {
                                    *a = *b;
                                }
                            }
                            (sum_a + sum_b, col_a)
                        },
                    );
                let sum_col_max: f64 = col_max.into_iter().sum();
                Ok((sum_row_max + sum_col_max) / total)
            } else {
                let mut col_max = vec![0.0_f64; terms2.len()];
                let mut sum_row_max: f64 = 0.0;

                for id1 in terms1 {
                    let mut row_max: f64 = 0.0;
                    for (j, id2) in terms2.iter().enumerate() {
                        let s = sim_fn.compute(id1, id2, counter);
                        if s > row_max {
                            row_max = s;
                        }
                        if s > col_max[j] {
                            col_max[j] = s;
                        }
                    }
                    sum_row_max += row_max;
                }

                let sum_col_max: f64 = col_max.into_iter().sum();
                Ok((sum_row_max + sum_col_max) / total)
            }
        }
        "avg" => {
             let count = (terms1.len() * terms2.len()) as f64;
             if count == 0.0 {
                 return Ok(0.0);
             }
             // sum( sim(t1, t2) ) / (N*M)
             let total_sim: f64 = if use_parallel {
                 terms1
                     .par_iter()
                     .map(|id1| {
                         terms2
                             .iter()
                             .map(|id2| sim_fn.compute(id1, id2, counter))
                             .sum::<f64>()
                     })
                     .sum()
             } else {
                 let mut total = 0.0;
                 for id1 in terms1 {
                     for id2 in terms2 {
                         total += sim_fn.compute(id1, id2, counter);
                     }
                 }
                 total
             };
                 
             Ok(total_sim / count)
        }
        "hausdorff" => {
            // min( min_a max_b sim(a, b), min_b max_a sim(b, a) )

            let (min_row_max, col_max) = if use_parallel {
                terms1
                    .par_iter()
                    .fold(
                        || (f64::INFINITY, vec![0.0_f64; terms2.len()]),
                        |(min_row, mut col_max), id1| {
                            let mut row_max: f64 = 0.0;
                            for (j, id2) in terms2.iter().enumerate() {
                                let s = sim_fn.compute(id1, id2, counter);
                                if s > row_max {
                                    row_max = s;
                                }
                                if s > col_max[j] {
                                    col_max[j] = s;
                                }
                            }
                            (min_row.min(row_max), col_max)
                        },
                    )
                    .reduce(
                        || (f64::INFINITY, vec![0.0_f64; terms2.len()]),
                        |(min_a, mut col_a), (min_b, col_b)| {
                            for (a, b) in col_a.iter_mut().zip(col_b.iter()) {
                                if *b > *a {
                                    *a = *b;
                                }
                            }
                            (min_a.min(min_b), col_a)
                        },
                    )
            } else {
                let mut col_max = vec![0.0_f64; terms2.len()];
                let mut min_row_max = f64::INFINITY;

                for id1 in terms1 {
                    let mut row_max: f64 = 0.0;
                    for (j, id2) in terms2.iter().enumerate() {
                        let s = sim_fn.compute(id1, id2, counter);
                        if s > row_max {
                            row_max = s;
                        }
                        if s > col_max[j] {
                            col_max[j] = s;
                        }
                    }
                    min_row_max = min_row_max.min(row_max);
                }

                (min_row_max, col_max)
            };

            let min_col_max: f64 = col_max.into_iter().fold(f64::INFINITY, f64::min);

            if min_row_max.is_infinite() || min_col_max.is_infinite() {
                Ok(0.0)
            } else {
                Ok(min_row_max.min(min_col_max))
            }
        }
        _ => Err(pyo3::exceptions::PyValueError::new_err(format!("Unknown groupwise strategy: {}", groupwise))),
    }
}

/// Internal helper to compute similarity between two sets of GO terms.
pub(crate) fn termset_similarity_internal(
    terms1: &[String],
    terms2: &[String],
    similarity: &str,
    groupwise: &str,
    counter: &TermCounter,
    ontology_terms: &HashMap<String, GOTerm>,
) -> PyResult<f64> {
    let sim_fn = if groupwise == "simgic" {
        None
    } else {
        Some(
            SimilarityMethod::from_str(similarity)
                .ok_or_else(|| PyValueError::new_err(format!("Unknown similarity method: {}", similarity)))?
        )
    };
    termset_similarity_internal_with_method(terms1, terms2, sim_fn, groupwise, counter, ontology_terms)
}


/// Compute semantic similarity between two sets of GO terms.
///
/// Arguments
/// ---------
/// terms1 : list of str
///   First list of GO term IDs.
/// terms2 : list of str
///   Second list of GO term IDs.
/// term_similarity : str
///   Name of the pairwise similarity method.
/// groupwise : str
///   Groupwise combination method. Options: "bma", "max", "avg", "hausdorff", "simgic".
/// counter : TermCounter
///   Precomputed IC values.
///
/// Returns
/// -------
/// float
///   Similarity score.
#[pyfunction]
#[pyo3(signature = (terms1, terms2, term_similarity="lin", groupwise="bma", counter=None))]
pub fn termset_similarity(
    terms1: Vec<String>,
    terms2: Vec<String>,
    term_similarity: &str,
    groupwise: &str,
    counter: Option<&TermCounter>,
) -> PyResult<f64> {
     let c = counter.ok_or_else(|| PyValueError::new_err("counter argument is required"))?;
     let terms_lock = get_terms_or_error()?;
     termset_similarity_internal(&terms1, &terms2, term_similarity, groupwise, c, &terms_lock)
}


In [ ]:
use pyo3::exceptions::PyValueError;
use pyo3::prelude::*;
use rayon::prelude::*;
use rustc_hash::{FxHashMap as HashMap, FxHashSet as HashSet};

use crate::go_loader::TermCounter;
use crate::go_ontology::{get_gene2go_or_error, get_terms_or_error};

use super::similarity::SimilarityMethod;
use super::termset::{termset_similarity_internal, termset_similarity_internal_with_method};

#[derive(Debug, Clone, Copy, PartialEq)]
enum DistanceTransform {
    OneMinus,
    Reciprocal,
    MaxMinus,
}

fn is_valid_groupwise(groupwise: &str) -> bool {
    matches!(groupwise, "bma" | "max" | "avg" | "hausdorff" | "simgic")
}

fn is_normalized_similarity(similarity: &str, groupwise: &str) -> bool {
    if groupwise == "simgic" {
        return true;
    }
    matches!(
        similarity.to_ascii_lowercase().as_str(),
        "lin" | "wang" | "simrel" | "topoicsim"
    )
}

fn resolve_distance_transform(
    distance_transform: &str,
    similarity: &str,
    groupwise: &str,
) -> PyResult<DistanceTransform> {
    match distance_transform.to_ascii_lowercase().as_str() {
        "auto" => {
            if is_normalized_similarity(similarity, groupwise) {
                Ok(DistanceTransform::OneMinus)
            } else {
                Ok(DistanceTransform::MaxMinus)
            }
        }
        "one_minus" | "one-minus" | "1-sim" | "1_minus" => Ok(DistanceTransform::OneMinus),
        "reciprocal" | "inv" | "inverse" => Ok(DistanceTransform::Reciprocal),
        "max_minus" | "max-minus" | "max" => Ok(DistanceTransform::MaxMinus),
        _ => Err(PyValueError::new_err(format!(
            "Unknown distance_transform '{}'. Options: auto, one_minus, reciprocal, max_minus",
            distance_transform
        ))),
    }
}

fn ontology_namespace(ontology: &str) -> PyResult<&'static str> {
    match ontology.to_ascii_uppercase().as_str() {
        "BP" => Ok("biological_process"),
        "MF" => Ok("molecular_function"),
        "CC" => Ok("cellular_component"),
        _ => Err(PyValueError::new_err(format!(
            "Invalid ontology '{}'. Must be 'BP', 'MF', or 'CC'",
            ontology
        ))),
    }
}

/// Compute semantic similarity between genes.
///
/// Arguments
/// ---------
/// gene1 : str
///   Gene symbol of the first gene.
/// gene2 : str
///   Gene symbol of the second gene.
/// ontology : str
///   Name of the subontology of GO to use: BP, MF or CC.
/// similarity : str
///   Name of the similarity method.
/// groupwise : str
///   Combination method to generate the similarities between genes. Options: "bma", "max", "avg", "hausdorff", "simgic".
/// counter : TermCounter
///   Precomputed IC values.
///
/// Returns
/// -------
/// float
///   Similarity score.
///
/// Raises
/// ------
/// ValueError
///   If method or combine are unknown.
#[pyfunction]
pub fn compare_genes(
    gene1: &str,
    gene2: &str,
    ontology: String,
    similarity: &str,
    groupwise: String,
    counter: &TermCounter,
) -> PyResult<f64> {
    let terms = get_terms_or_error()?;
    let gene2go = get_gene2go_or_error()?;
    let g1_terms = gene2go.get(gene1).ok_or_else(|| {
        pyo3::exceptions::PyValueError::new_err(format!("Gene '{}' not found in mapping", gene1))
    })?;
    let g2_terms = gene2go.get(gene2).ok_or_else(|| {
        pyo3::exceptions::PyValueError::new_err(format!("Gene '{}' not found in mapping", gene2))
    })?;
    let ns = ontology_namespace(&ontology)?;
    let f1: Vec<String> = g1_terms
        .iter()
        .filter(|id| terms.get(*id).map_or(false, |t| t.namespace.to_ascii_lowercase() == ns))
        .cloned()
        .collect();

    let f2: Vec<String> = g2_terms
        .iter()
        .filter(|id| terms.get(*id).map_or(false, |t| t.namespace.to_ascii_lowercase() == ns))
        .cloned()
        .collect();

    if f1.is_empty() || f2.is_empty() {
        return Ok(0.0);
    }
 
    termset_similarity_internal(&f1, &f2, similarity, &groupwise, counter, &terms)
}

/// Compute semantic similarity between genes in batches.
///
/// Arguments
/// ---------
/// pairs : list of (str, str)
///   List of pairs of genes to calculate the semantic similarity
/// ontology : str
///   Name of the subontology of GO to use: BP, MF or CC.
/// similarity : str
///   Name of the similarity method.
/// groupwise : str
///   Combination method to generate the similarities between genes. Options: "bma", "max", "avg", "hausdorff", "simgic".
/// counter : TermCounter
///   Precomputed IC values.
///
/// Returns
/// -------
/// list of float
///   List of similarity scores.
///
/// Raises
/// ------
/// ValueError
///   If method or combine are unknown.
#[pyfunction]
#[pyo3(signature = (pairs, ontology, similarity, groupwise, counter))]
pub fn compare_gene_pairs_batch(
    pairs: Vec<(String, String)>,
    ontology: String,
    similarity: &str,
    groupwise: String,
    counter: &TermCounter,
) -> PyResult<Vec<f64>> {
    let gene2go = get_gene2go_or_error()?;
    let terms = get_terms_or_error()?;
    let ns = ontology_namespace(&ontology)?;
    let sim_fn = if groupwise == "simgic" {
        None
    } else {
        SimilarityMethod::from_str(similarity)
    };

    let mut unique_genes: HashSet<String> = HashSet::default();
    for (g1, g2) in &pairs {
        unique_genes.insert(g1.clone());
        unique_genes.insert(g2.clone());
    }

    let gene_terms: HashMap<String, Vec<String>> = unique_genes
        .into_iter()
        .map(|gene| {
            let filtered: Vec<String> = gene2go
                .get(&gene)
                .into_iter()
                .flatten()
                .filter(|go| {
                    terms
                        .get(go.as_str())
                        .map_or(false, |t| t.namespace.eq_ignore_ascii_case(ns))
                })
                .cloned()
                .collect();
            (gene, filtered)
        })
        .collect();

    let empty: Vec<String> = Vec::new();
    let scores: Vec<f64> = pairs
        .par_iter()
        .map(|(g1, g2)| {
            let go1 = gene_terms.get(g1).unwrap_or(&empty);
            let go2 = gene_terms.get(g2).unwrap_or(&empty);

            if go1.is_empty() || go2.is_empty() {
                return 0.0;
            }

            termset_similarity_internal_with_method(go1, go2, sim_fn, &groupwise, counter, &terms)
                .unwrap_or(0.0)
        })
        .collect();

    Ok(scores)
}

/// Compute a gene-to-gene distance matrix using GO semantic similarity.
///
/// Arguments
/// ---------
/// genes : Optional[list[str]]
///   List of genes to include. If None, uses all genes with annotations.
/// ontology : str
///   Name of the subontology of GO to use: BP, MF or CC.
/// similarity : str
///   Name of the similarity method.
/// groupwise : str
///   Combination method to generate the similarities between genes. Options: "bma", "max", "avg", "hausdorff", "simgic".
/// counter : TermCounter
///   Precomputed IC values.
/// distance_transform : str
///   How to convert similarity to distance. Options: "auto", "one_minus", "reciprocal", "max_minus".
///
/// Returns
/// -------
/// (list[str], list[list[float]])
///   Tuple with the gene order and a square distance matrix.
#[pyfunction]
#[pyo3(signature = (genes=None, ontology="BP", similarity="lin", groupwise="bma", counter=None, distance_transform="auto"))]
pub fn gene_distance_matrix(
    genes: Option<Vec<String>>,
    ontology: &str,
    similarity: &str,
    groupwise: &str,
    counter: Option<&TermCounter>,
    distance_transform: &str,
) -> PyResult<(Vec<String>, Vec<Vec<f64>>)> {
    let counter = counter.ok_or_else(|| PyValueError::new_err("counter argument is required"))?;
    if !is_valid_groupwise(groupwise) {
        return Err(PyValueError::new_err(format!(
            "Unknown groupwise strategy: {}",
            groupwise
        )));
    }

    let terms = get_terms_or_error()?;
    let gene2go = get_gene2go_or_error()?;
    let ns = ontology_namespace(ontology)?;

    let gene_list = match genes {
        Some(list) => list,
        None => {
            let mut all: Vec<String> = gene2go.keys().cloned().collect();
            all.sort();
            all
        }
    };

    if gene_list.is_empty() {
        return Ok((gene_list, Vec::new()));
    }

    let missing: Vec<String> = gene_list
        .iter()
        .filter(|g| !gene2go.contains_key(*g))
        .cloned()
        .collect();
    if !missing.is_empty() {
        return Err(PyValueError::new_err(format!(
            "Genes not found in mapping: {}",
            missing.join(", ")
        )));
    }

    let sim_fn = if groupwise == "simgic" {
        None
    } else {
        Some(
            SimilarityMethod::from_str(similarity)
                .ok_or_else(|| PyValueError::new_err(format!("Unknown similarity method: {}", similarity)))?
        )
    };

    let gene_terms: Vec<Vec<String>> = gene_list
        .par_iter()
        .map(|gene| {
            let terms_for_gene = gene2go.get(gene).unwrap();
            terms_for_gene
                .iter()
                .filter(|go| {
                    terms
                        .get(go.as_str())
                        .map_or(false, |t| t.namespace.eq_ignore_ascii_case(ns))
                })
                .cloned()
                .collect()
        })
        .collect();

    let n = gene_list.len();
    let mut matrix = vec![vec![0.0; n]; n];

    // Parallelize over all (i, j) in the upper triangle (including diagonal) to improve
    // load balancing across threads. This is especially important when each pairwise gene
    // comparison is expensive (large term sets / slower similarity methods).
    let pairs: Vec<(usize, usize)> = (0..n)
        .flat_map(|i| (i..n).map(move |j| (i, j)))
        .collect();

    let sims: Vec<f64> = pairs
        .par_iter()
        .map(|(i, j)| {
            termset_similarity_internal_with_method(
                &gene_terms[*i],
                &gene_terms[*j],
                sim_fn,
                groupwise,
                counter,
                &terms,
            )
            .unwrap_or(0.0)
        })
        .collect();

    for ((i, j), sim) in pairs.into_iter().zip(sims.into_iter()) {
        matrix[i][j] = sim;
        matrix[j][i] = sim;
    }

    let transform = resolve_distance_transform(distance_transform, similarity, groupwise)?;
    match transform {
        DistanceTransform::MaxMinus => {
            let mut max_sim = 0.0;
            for row in &matrix {
                for &v in row {
                    if v > max_sim {
                        max_sim = v;
                    }
                }
            }
            matrix.par_iter_mut().for_each(|row| {
                for v in row.iter_mut() {
                    let d = max_sim - *v;
                    *v = if d < 0.0 { 0.0 } else { d };
                }
            });
        }
        DistanceTransform::OneMinus => {
            matrix.par_iter_mut().for_each(|row| {
                for v in row.iter_mut() {
                    let d = 1.0 - *v;
                    *v = if d < 0.0 { 0.0 } else { d };
                }
            });
        }
        DistanceTransform::Reciprocal => {
            matrix.par_iter_mut().for_each(|row| {
                for v in row.iter_mut() {
                    *v = 1.0 / (1.0 + *v);
                }
            });
        }
    }

    for i in 0..n {
        matrix[i][i] = 0.0;
    }

    Ok((gene_list, matrix))
}
